In [2]:
import re
from bs4 import BeautifulSoup
import datasets

In [3]:

ddata=datasets.load_dataset('../datasets/nq_3kl')
# data=datasets.load_dataset('json',data_files='nq_3kl/1_3_NQ_10k_multi_task_train.json')

Using custom data configuration nq_3kl-1ddbf9404c7ea467
Reusing dataset json (/home/adminroot/.cache/huggingface/datasets/json/nq_3kl-1ddbf9404c7ea467/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
ddata=ddata['train']
ddata[0]['text']
#has been parse the html to the normal text

"document:Mini ( marque ) - wikipedia  Mini ( marque )    For other uses of `` mini '' , see mini ( disambiguation ) .    Mini       Product type   Automobiles     Owner   BMW ( directly since 2000 )     Produced by   BMW     Country   United Kingdom     Introduced   1969 ( used since 1959 in model names )     Markets   Worldwide     Previous owners     British Motor Corporation ( 1959 - 1968 )   British Leyland ( 1968 - 1986 )   Rover Group ( 1986 - 2000 )       Website   mini.com     Mini ( stylised as MINI ) is a British automotive marque , owned by BMW since 2000 , and used by them for a range of small cars . The word Mini has been used in car model names since 1959 , and in 1969 it became a marque in its own right when the name `` Mini '' replaced the separate `` Austin Mini '' and `` Morris Mini '' car model names . BMW acquired the marque in 1994 when it bought Rover Group ( formerly British Leyland ) , which owned Mini , among other brands .   The original Mini was a line of ic

In [5]:
soup=BeautifulSoup(ddata[0]['text'],'html.parser')
print(soup.get_text())
print(len(soup.get_text()))

document:Mini ( marque ) - wikipedia  Mini ( marque )    For other uses of `` mini '' , see mini ( disambiguation ) .    Mini       Product type   Automobiles     Owner   BMW ( directly since 2000 )     Produced by   BMW     Country   United Kingdom     Introduced   1969 ( used since 1959 in model names )     Markets   Worldwide     Previous owners     British Motor Corporation ( 1959 - 1968 )   British Leyland ( 1968 - 1986 )   Rover Group ( 1986 - 2000 )       Website   mini.com     Mini ( stylised as MINI ) is a British automotive marque , owned by BMW since 2000 , and used by them for a range of small cars . The word Mini has been used in car model names since 1959 , and in 1969 it became a marque in its own right when the name `` Mini '' replaced the separate `` Austin Mini '' and `` Morris Mini '' car model names . BMW acquired the marque in 1994 when it bought Rover Group ( formerly British Leyland ) , which owned Mini , among other brands .   The original Mini was a line of ico

In [6]:
len(ddata)

6750

In [7]:
print(ddata[0]['text'][9:])

Mini ( marque ) - wikipedia  Mini ( marque )    For other uses of `` mini '' , see mini ( disambiguation ) .    Mini       Product type   Automobiles     Owner   BMW ( directly since 2000 )     Produced by   BMW     Country   United Kingdom     Introduced   1969 ( used since 1959 in model names )     Markets   Worldwide     Previous owners     British Motor Corporation ( 1959 - 1968 )   British Leyland ( 1968 - 1986 )   Rover Group ( 1986 - 2000 )       Website   mini.com     Mini ( stylised as MINI ) is a British automotive marque , owned by BMW since 2000 , and used by them for a range of small cars . The word Mini has been used in car model names since 1959 , and in 1969 it became a marque in its own right when the name `` Mini '' replaced the separate `` Austin Mini '' and `` Morris Mini '' car model names . BMW acquired the marque in 1994 when it bought Rover Group ( formerly British Leyland ) , which owned Mini , among other brands .   The original Mini was a line of iconic Briti

In [8]:
print(ddata[1])

{'doc_id': '179877', 'text': 'question:when did the cooper s get a turbo'}


In [9]:
print(ddata[0]['doc_id']==ddata[1]['doc_id'])

True


In [10]:
from transformers import *
current_path='./model/bert-base-uncased/'
# model=BertModel.from_pretrained(current_path)
# tokenizer=BertTokenizer.from_pretrained(current_path)
model=AutoModelWithLMHead.from_pretrained(current_path)
tokenizer=AutoTokenizer.from_pretrained(current_path)


/home/adminroot/miniconda3/envs/pc_demo1/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:878: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file ./model/bert-base-uncased/config.json
Model config BertConfig {
  "_name_or_path": "./model/bert-base-uncased/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "positi

In [11]:
tmptxt=ddata[0]['text'][9:]
tmptxt


"Mini ( marque ) - wikipedia  Mini ( marque )    For other uses of `` mini '' , see mini ( disambiguation ) .    Mini       Product type   Automobiles     Owner   BMW ( directly since 2000 )     Produced by   BMW     Country   United Kingdom     Introduced   1969 ( used since 1959 in model names )     Markets   Worldwide     Previous owners     British Motor Corporation ( 1959 - 1968 )   British Leyland ( 1968 - 1986 )   Rover Group ( 1986 - 2000 )       Website   mini.com     Mini ( stylised as MINI ) is a British automotive marque , owned by BMW since 2000 , and used by them for a range of small cars . The word Mini has been used in car model names since 1959 , and in 1969 it became a marque in its own right when the name `` Mini '' replaced the separate `` Austin Mini '' and `` Morris Mini '' car model names . BMW acquired the marque in 1994 when it bought Rover Group ( formerly British Leyland ) , which owned Mini , among other brands .   The original Mini was a line of iconic Brit

In [12]:
from torch.nn.utils.rnn import pad_sequence
import torch
import math
def txt_encode(tmptxt):
    tok_tmptxt=tokenizer.encode(tmptxt,add_special_tokens=False)
    tmp=[]
    if len(tok_tmptxt)>=510:
        tmp=[]
        for i in range(math.ceil(len(tok_tmptxt)/510)):
            tmp.append(tok_tmptxt[i*510:i*510+510])
    tmp=[pad_sequence(torch.tensor(i).unsqueeze(0),padding_value=tokenizer.pad_token_id) for i in tmp]
    return tmp

In [13]:
import math

pad_tmptxt1=txt_encode(ddata[0]['text'][9:])
# print(len(pad_tmptxt1[0]))
head_token=torch.tensor([101])
tail_token=torch.tensor([102])  
final_txt_list=[]
for i in pad_tmptxt1:
    t=torch.stack(tuple(i),1)
    final_t=torch.cat((head_token,t[0],tail_token))
    # print(len(final_t))
    final_txt_list.append(final_t)
    
print(final_txt_list[0])
print(len(final_txt_list[-1]))
    

# print(len(pad_tmptxt1[0]))
# print(pad_tmptxt1[0])
# pad_tmptxt1=torch.stack(tuple(txt_encode(ddata[0]['text'][9:])),1)
# pad_tmptxt2=txt_encode(ddata[1]['text'][9:])
# pad_tmptxt2=torch.stack(tuple(txt_encode(ddata[1]['text'][9:])),1)
# print(len(pad_tmptxt1))  #1
# tmp=[]
# print(len(pad_tmptxt1[0]))
# # print(math.ceil(len(pad_tmptxt1[0])//510))
# for i in range(math.ceil(len(pad_tmptxt1[0])/510)):
#     tmp.append(pad_tmptxt1[0][i*510:i*510+510])
# print(len(tmp[0]))


# print(len(pad_tmptxt1[0]))
# print(pad_tmptxt1)

tensor([  101,  7163,  1006,  9388,  4226,  1007,  1011, 16948,  7163,  1006,
         9388,  4226,  1007,  2005,  2060,  3594,  1997,  1036,  1036,  7163,
         1005,  1005,  1010,  2156,  7163,  1006,  4487, 21559,  5638, 19696,
         3508,  1007,  1012,  7163,  4031,  2828, 19207,  3954, 13154,  1006,
         3495,  2144,  2456,  1007,  2550,  2011, 13154,  2406,  2142,  2983,
         3107,  3440,  1006,  2109,  2144,  3851,  1999,  2944,  3415,  1007,
         6089,  4969,  3025,  5608,  2329,  5013,  3840,  1006,  3851,  1011,
         3380,  1007,  2329, 25866,  3122,  1006,  3380,  1011,  3069,  1007,
        13631,  2177,  1006,  3069,  1011,  2456,  1007,  4037,  7163,  1012,
         4012,  7163,  1006,  2358,  8516,  5084,  2004,  7163,  1007,  2003,
         1037,  2329, 12945,  9388,  4226,  1010,  3079,  2011, 13154,  2144,
         2456,  1010,  1998,  2109,  2011,  2068,  2005,  1037,  2846,  1997,
         2235,  3765,  1012,  1996,  2773,  7163,  2038,  2042, 

In [14]:
# model=model.module if hasattr(model,"module") else model
all_layer_hiddens=model.bert(torch.tensor(final_txt_list[0]).clone().detach().unsqueeze(0))
print(len(all_layer_hiddens[0]),len(all_layer_hiddens[0][0]),len(all_layer_hiddens[0][0][0]))

/tmp/ipykernel_1776901/2484704068.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_layer_hiddens=model.bert(torch.tensor(final_txt_list[0]).clone().detach().unsqueeze(0))


1 512 768


In [87]:
# model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [15]:
print(all_layer_hiddens[0].shape)
# print(all_layer_hiddens[0])
tens=all_layer_hiddens[0].squeeze()
# print(tens.squeeze())
print(tens.shape)
# torch.Size([1, 10, 768])
# torch.Size([10, 768])

torch.Size([1, 512, 768])
torch.Size([512, 768])


In [16]:
print(torch.mean(tens,0).detach())#tens torch.Size([10, 768])
print(len(torch.mean(tens,0).detach()))

tensor([-3.0111e-01, -1.8997e-02,  5.7799e-01, -1.4761e-01,  1.5393e-01,
         6.9918e-02,  7.8876e-02,  1.7602e-01,  3.2789e-01,  8.5336e-02,
        -5.6662e-02, -2.0416e-01, -1.3870e-02,  5.4862e-01, -1.5995e-01,
         1.9530e-01,  2.0476e-01,  2.0582e-01, -4.2941e-01,  3.7668e-01,
         4.9331e-01, -2.6728e-02,  1.7861e-02,  6.0671e-01,  2.9071e-01,
         1.6942e-01,  5.1485e-02, -4.4091e-03,  5.3772e-02, -9.6332e-03,
         4.9164e-01,  1.9461e-02, -2.4845e-01,  2.2925e-02,  2.4130e-01,
        -3.0984e-01, -7.4167e-02, -3.9742e-01, -1.6909e-01,  3.8087e-02,
        -5.6183e-01, -4.6291e-01, -2.4533e-02,  1.7299e-01, -1.3959e-01,
         1.7093e-02,  2.3695e-01,  6.6027e-02, -4.6363e-02,  2.4234e-01,
        -6.5622e-01,  1.8420e-01, -8.6174e-02, -5.5353e-03,  3.2440e-01,
         5.2500e-01,  1.0060e-02, -6.3154e-01, -3.4865e-01, -2.7658e-01,
         6.3590e-02,  1.9139e-01,  2.0197e-01, -1.9604e-01,  1.4397e-02,
         3.7949e-02, -2.3208e-02,  2.5717e-01, -1.0

In [17]:
cnt=0.0
re=0.0
for i in all_layer_hiddens[0][0]:
    # print(i[0])
    re+=i[0]
    cnt+=1
# print(all_layer_hiddens[0])
print(re/cnt)

tensor(-0.3011, grad_fn=<DivBackward0>)


In [18]:
# print((torch.mean(all_layer_hiddens[0],0)).shape)   #all_layer_size=torch.Size([1, 10, 768])
print(len((torch.mean(all_layer_hiddens[0],1).squeeze().detach())))
print((torch.mean(all_layer_hiddens[0],1).squeeze().detach()))

768
tensor([-3.0111e-01, -1.8997e-02,  5.7799e-01, -1.4761e-01,  1.5393e-01,
         6.9918e-02,  7.8876e-02,  1.7602e-01,  3.2789e-01,  8.5336e-02,
        -5.6662e-02, -2.0416e-01, -1.3870e-02,  5.4862e-01, -1.5995e-01,
         1.9530e-01,  2.0476e-01,  2.0582e-01, -4.2941e-01,  3.7668e-01,
         4.9331e-01, -2.6728e-02,  1.7861e-02,  6.0671e-01,  2.9071e-01,
         1.6942e-01,  5.1485e-02, -4.4091e-03,  5.3772e-02, -9.6332e-03,
         4.9164e-01,  1.9461e-02, -2.4845e-01,  2.2925e-02,  2.4130e-01,
        -3.0984e-01, -7.4167e-02, -3.9742e-01, -1.6909e-01,  3.8087e-02,
        -5.6183e-01, -4.6291e-01, -2.4533e-02,  1.7299e-01, -1.3959e-01,
         1.7093e-02,  2.3695e-01,  6.6027e-02, -4.6363e-02,  2.4234e-01,
        -6.5622e-01,  1.8420e-01, -8.6174e-02, -5.5353e-03,  3.2440e-01,
         5.2500e-01,  1.0060e-02, -6.3154e-01, -3.4865e-01, -2.7658e-01,
         6.3590e-02,  1.9139e-01,  2.0197e-01, -1.9604e-01,  1.4397e-02,
         3.7949e-02, -2.3208e-02,  2.5717e-01, 

In [ ]:
#abstract by bart to 512 or mean(len/512)


In [24]:
import json
from tqdm import trange
#set doc_id,document_txt,questions_txt
head_token=torch.tensor([101])
tail_token=torch.tensor([102])  
with open('1_3_nq_embeddings.json','w') as tf:
    for ind in trange(len(ddata)):
        docid=ddata[ind]['doc_id']
        if ind-1>=0 and ddata[ind-1]['doc_id']==ddata[ind]['doc_id']:
            text=ddata[ind-1]['text'][9:]+' '+ddata[ind]['text'][9:]
            #convert txt to embeddings
            pad_tmptxt1=txt_encode(text)
            all_layer_hiddens_list=[]
            for i in pad_tmptxt1:
                t=torch.stack(tuple(i),1)
                final_t=torch.cat((head_token,t[0],tail_token))
                a=model.bert(torch.tensor(final_t).clone().detach().unsqueeze(0)) #[1,512,786]
                all_layer_hiddens_list.append(torch.mean(a[0].squeeze(0),0))
            if all_layer_hiddens_list:
                doc_vector=torch.mean(torch.stack(all_layer_hiddens_list,0),0)
            tf.write(json.dumps({"id":str(docid),"vector":doc_vector.detach().numpy().tolist()})+'\n')



  0%|          | 0/6750 [00:00<?, ?it/s]

/tmp/ipykernel_1776901/80789091.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a=model.bert(torch.tensor(final_t).clone().detach().unsqueeze(0)) #[1,512,786]
100%|██████████| 6750/6750 [1:10:37<00:00,  1.59it/s] 
